In [1]:
%load_ext autoreload
%autoreload 2
# %matplotlib inline

# Load Foil Data

In [2]:
from ICARUS.Software.XFLR5.polars import readPolars2D
from ICARUS.Database.db import DB
from ICARUS.Database import XFLRDB

In [3]:
db = DB()
foildb = db.foilsDB
foildb.loadData()
readPolars2D(foildb,XFLRDB)
airfoils = foildb.getAirfoils()

# Get Plane

In [ ]:
from Data.Planes.hermes import hermes

In [ ]:
airplane = hermes(airfoils ,"Hermes")

# Import Enviroment

In [ ]:
from ICARUS.Enviroment.definition import EARTH
print(EARTH)

# Get Solver

In [ ]:
from ICARUS.Software.GenuVP3.gnvp3 import get_gnvp3
gnvp3 = get_gnvp3(db)
print(gnvp3)

## AoA Run

In [ ]:
analysis = gnvp3.getAvailableAnalyses()[2] # ANGLES PARALLEL
print(f"Selecting Analysis: {analysis}")
gnvp3.setAnalysis(analysis)
options = gnvp3.getOptions(analysis)
solver_parameters = gnvp3.getSolverParameters(analysis)

In [ ]:
AoAmin = -6
AoAmax = 8
NoAoA = (AoAmax - AoAmin) + 1
angles = np.linspace(AoAmin, AoAmax, NoAoA)
u_freestream = 20
airplane.defineSim(u_freestream, EARTH.AirDensity)

options.plane.value         = airplane
options.environment.value   = EARTH
options.db.value            = db
options.solver2D.value      = 'XFLR'
options.maxiter.value       = 100
options.timestep.value      = 5e-2
options.u_freestream.value          = u_freestream
options.angles.value        = angles

gnvp3.printOptions()
gnvp3.run()
polars = gnvp3.getResults()
airplane.save()

# Dynamics

### Define and Trim Plane

In [ ]:
from ICARUS.Flight_Dynamics.dyn_plane import dyn_Airplane as dp
dyn = dp(airplane,polars)

### Pertrubations

In [ ]:
dyn.allPerturb("Central")
# dyn.get_pertrub()

In [ ]:
analysis = gnvp3.getAvailableAnalyses()[4] # ANGLES PARALLEL
print(f"Selecting Analysis: {analysis}")
gnvp3.setAnalysis(analysis)
options = gnvp3.getOptions(analysis)

In [ ]:
dyn.defineSim(dyn.trim['U'], EARTH.AirDensity)

options.plane.value         = dyn
options.environment.value   = EARTH
options.db.value            = db
options.solver2D.value      = 'XFLR'
options.maxiter.value       = 200
options.timestep.value      = 5e-3
options.u_freestream.value          = dyn.trim['U']
options.angles.value        = dyn.trim['AoA']

gnvp3.printOptions()
gnvp3.run()
gnvp3.getResults()
dyn.save()

# Sensitivity Analysis

In [ ]:
# if sensAnalysisGNVP3 :
#     for var in ['u','w','q','theta','v','p','r','phi']:
#         space = np.logspace(np.log10(0.00001), np.log10(1), 10, base=10)
#         space =[*-space,*space]
#         maxiter = 2
#         timestep = 5e-2
#         dyn.sensitivityAnalysis(var, space)
#         GNVP3BatchArgs = [dyn, var, foildb, "Xfoil",
#                         maxiter, timestep,
#                         dyn.trim['U'], dyn.trim['AoA']]
#         dyn.runAnalysis(runGNVPsensitivityParallel, GNVP3BatchArgs)
#         perDIR = os.path.join(dyn.CASEDIR, f"Sensitivity_{var}")
#         dyn.sensResults[var] = pertrResults(perDIR, HOMEDIR)
#     dyn.save()